In [267]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\fefif\Desktop\PhD\Progetti\MatrixProductBP.jl`


In [268]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
include("bp_fourier.jl");

In [269]:
T = 4
N = 5
k = 4
seed = 3
rng = MersenneTwister(seed)
A = [0 1 1 1 1; 1 0 0 0 0; 1 0 0 0 0; 1 0 0 0 0; 1 0 0 0 0]
g = IndexedBiDiGraph(A)
# rand!(rng, A.nzval, (-1,1))  # fill directed incidence matrix with rand(-1,1)
β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(100)

A

5×5 Matrix{Int64}:
 0  1  1  1  1
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0

In [270]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
iterate!(bp, maxiter=1; svd_trunc)
msg = deepcopy(bp.μ)
jldsave("messages.jld2"; msg)

bp_fourier = deepcopy(bp)
;

In [271]:
iterate!(bp,maxiter=1; svd_trunc)
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/50; svd_trunc)
;

In [272]:
node = 2
time = 5

belief = bp.b[node]
belief_fourier = bp_fourier.b[node]

marg = marginals(belief)[time]
marg_fourier = marginals(belief_fourier)[time]

display(marg)
display(marg_fourier);

2-element Vector{Float64}:
 0.1538555459146213
 0.8461444540853787

2-element Vector{Float64}:
 0.38174156831313777
 0.6182584316868622

In [273]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [274]:
node = 2
time = 5

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

display(marg)
display(marg_fourier);

ed = Indexed Edge 1 => 2 with index 1
ed_fourier = Indexed Edge 1 => 2 with index 1


2-element Vector{Float64}:
 0.1920663094856764
 0.8079336905143235

2-element Vector{Float64}:
 0.45301658137141904
 0.5469834186285809

In [275]:
msg[1][2]

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.240205   0.136236
  0.237768  -0.134854

[:, :, 2, 1] =
 -0.240567  0.0607763
 -0.243032  0.061399

[:, :, 1, 2] =
 -0.272441  -0.0523488
  0.269678   0.0518178

[:, :, 2, 2] =
 -0.271709  -0.121402
 -0.274493  -0.122646

In [276]:
msg_fourier = FourierTensorTrain_spin(msg[1], K, 1, 2.0, 1/50)
msg_fourier[2]

2×2×201×2 OffsetArray(::Array{ComplexF64, 4}, 1:2, 1:2, -100:100, 1:2) with eltype ComplexF64 with indices 1:2×1:2×-100:100×1:2:
[:, :, -100, 1] =
 -6.32355e-18-9.37142e-36im   2.59128e-18-1.94968e-33im
 -6.92377e-20-1.24226e-32im  -9.66146e-19+5.07067e-33im

[:, :, -99, 1] =
 1.38724e-17+1.33385e-34im  -5.68467e-18+2.77502e-32im
 1.51891e-19+1.76814e-31im    2.1195e-18-7.21719e-32im

[:, :, -98, 1] =
 -3.01935e-17+7.5894e-34im    1.23728e-17+1.57894e-31im
 -3.30594e-19+1.00604e-30im  -4.61312e-18-4.10645e-31im

;;; … 

[:, :, 98, 1] =
 -3.01935e-17-7.5894e-34im    1.23728e-17-1.57894e-31im
 -3.30594e-19-1.00604e-30im  -4.61312e-18+4.10645e-31im

[:, :, 99, 1] =
 1.38724e-17-1.33385e-34im  -5.68467e-18-2.77502e-32im
 1.51891e-19-1.76814e-31im    2.1195e-18+7.21719e-32im

[:, :, 100, 1] =
 -6.32355e-18+9.37142e-36im   2.59128e-18+1.94968e-33im
 -6.92377e-20+1.24226e-32im  -9.66146e-19-5.07067e-33im

[:, :, -100, 2] =
 -7.15717e-18+1.89219e-35im  -2.28533e-18-1.78416e-33im
 -6.33323e-20-

In [277]:
msg_fourier[2][:,:,-1,:]

2×2×2 OffsetArray(::Array{ComplexF64, 3}, 1:2, 1:2, 1:2) with eltype ComplexF64 with indices 1:2×1:2×1:2:
[:, :, 1] =
   0.880047-8.13081e-20im  -0.360628-1.69157e-17im
 0.00963579-1.07781e-16im   0.134458+4.3994e-17im

[:, :, 2] =
    0.99606+1.6417e-19im   0.318049-1.54797e-17im
 0.00881393-1.21987e-16im   0.12965-3.91096e-17im